In [1]:
%matplotlib inline
from __future__ import division
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import re
import math
from adjustText import adjust_text

import common
import notebook_helpers

FS = (10, 7)

om = notebook_helpers.get_frame()
lyrs = lyr = notebook_helpers.get_frame(having_lyrics=True)

In [3]:
om[om['artist'] == 'Paul Simon']

,artist,comp,date,icomp,peak,ratio,raw,scraped,title,genre,raw_ratio,unbiased_ratio,year,yearf,rscore
23787,Paul Simon,594.0,1973-12-01,871.0,35,1.425947,1242.0,True,American Tune,None,2.090909,2.160000,1973,1974.002740,0.511921
23788,Paul Simon,566.0,1987-03-07,800.0,86,2.190000,1752.0,True,The Boy In The Bubble,None,3.095406,3.202925,1987,1987.269178,1.130931
23789,Paul Simon,405.0,1973-05-19,557.0,2,2.150808,1198.0,True,Kodachrome,None,2.958025,3.103627,1973,1973.468721,1.104879
23790,Paul Simon,406.0,1976-05-01,565.0,40,1.447788,818.0,True,Still Crazy After All These Years,None,2.014778,2.113695,1976,1976.419406,0.533850
23791,Paul Simon,436.0,1983-11-05,627.0,44,1.422648,892.0,True,Allergies,None,2.045872,2.139089,1983,1983.930365,0.508578
23792,Paul Simon,742.0,1972-07-08,1093.0,52,1.273559,1392.0,True,Duncan,None,1.876011,1.925311,1972,1972.605251,0.348866
23793,Paul Simon,NaN,1980-10-25,NaN,40,NaN,NaN,False,One-Trick Pony,None,NaN,NaN,1980,1980.901826,NaN
23794,Paul Simon,456.0,1990-12-15,640.0,92,1.493750,956.0,True,The Obvious Child,None,2.096491,2.187643,1990,1991.041096,0.578939
23795,Paul Simon,688.0,1986-12-06,994.0,81,1.675050,1665.0,True,Graceland,None,2.420058,2.488789,1986,1987.016438,0.744204
23796,Paul Simon,571.0,1980-08-09,830.0,6,1.412048,1172.0,True,Late In The Evening,None,2.052539,2.123188,1980,1980.691324,0.497789


In [2]:
import json
def to_js(df, fname, varname="DATA", pprint=1):
    json_str = df.to_json(path_or_buf=None, orient='records')
    if pprint:
        json_str = json_str.replace('},', '},\n  ')
    _json_str_to_js(json_str, fname, varname)
    
def dict_to_js(d, fname, varname="DATA", pprint=1):
    s = json.dumps(d, indent=(2 if pprint else None))
    _json_str_to_js(s, fname, varname)
    
def _json_str_to_js(json_str, fname, varname):
    if not fname.endswith('.js'):
        fname += '.js'
    with open(fname, 'w') as f:
        f.write('var {} = '.format(varname))
        f.write(json_str)
        f.write(';\nexport default {};'.format(varname))
    print "Wrote results to {}".format(fname)
    os.system('cp {} ~/src/song-repetition/src/js/'.format(fname))

In [11]:
disco_dir = 'discogs'
final_dest = '~/src/song-repetition/src/assets/'
def artist_to_fname(a):
    slug = a.replace(' ', '_').replace('/', '').replace("'", "").lower() + '_discog'
    return slug + '.json'

songs_per_artist = lyrs.groupby('artist').size()
stars = songs_per_artist.to_frame(name='nsongs')
istar = stars.index
stars['year'] = lyrs.groupby('artist')['year'].median().loc[istar].astype(int)

irealstar = (
    (stars['year'] > 2000) & (stars['nsongs'] > 4)
    |
    (stars['year'] > 1990) & (stars['nsongs'] > 7)
    |
    (stars['year'] > 1980) & (stars['nsongs'] > 10)
    |
    (stars['year'] > 1980) & (stars['nsongs'] > 15)
)

stars = stars.loc[irealstar]

stars['name'] = stars.index.to_series()
stars['fname'] = stars['name'].apply(artist_to_fname)
foo = stars['fname'].head().to_dict()
foo

{u'10,000 Maniacs': u'10,000_maniacs_discog.json',
 u'2Pac': u'2pac_discog.json',
 u'3 Doors Down': u'3_doors_down_discog.json',
 u'3OH!3': u'3oh!3_discog.json',
 u'5 Seconds Of Summer': u'5_seconds_of_summer_discog.json'}

In [9]:
# Numbers for the artist discography widget (to be pasted in to source)
print "Mean rscore: {:.3f}".format(lyrs['rscore'].mean())
print "Mean of compression %: {:1%}".format(lyrs['rscore'].apply(lambda rs: 2**-rs).mean())
print "Harmonic mean: {:1%}".format(2**(-1 * lyrs['rscore'].mean()))
lyrs['rscore'].describe([.001, .005, .01, .1,.25,.5,.75,.9, .95, .98, .99, .995, .999])

Mean rscore: 0.995
Mean of compression %: 51.785706%
Harmonic mean: 50.183919%


count    14760.000000
mean         0.994703
std          0.371686
min          0.079434
0.1%         0.181736
0.5%         0.258959
1%           0.298361
10%          0.538801
25%          0.734790
50%          0.973328
75%          1.215499
90%          1.467091
95%          1.630351
98%          1.847015
99%          1.999676
99.5%        2.174265
99.9%        2.695442
max          5.419097
Name: rscore, dtype: float64

In [27]:
lyrs['unbiased_ratio'].describe([.1,.25,.5,.75,.9])

count    14760.000000
mean         2.951639
std          0.859311
min          1.250000
10%          2.125389
25%          2.422644
50%          2.833211
75%          3.314685
90%          3.879962
max         47.454545
Name: unbiased_ratio, dtype: float64

In [31]:
print lyrs['year'].mean()
lyrs.sort_values(by='rscore', ascending=0).head(10)['year'].mean()

1990.02479675


1995.0999999999999

In [25]:
cols = ['artist', 'title', 'rscore']
lyrs[
    (lyrs['year']==2016)
    & (lyrs['peak'] < 40)
][cols].sample(10)

,artist,title,rscore
1040,Little Big Town,Better Man,1.005159
11053,Coldplay,Hymn For The Weekend,1.463531
980,DJ Snake Featuring Justin Bieber,Let Me Love You,1.600248
14677,Rihanna Featuring Drake,Work,1.006650
79,Kungs vs Cookin' On 3 Burners,This Girl,0.769451
22259,DJ Khaled Featuring JAY Z & Future,I Got The Keys,1.181263
22339,Selena Gomez,Kill Em With Kindness,1.444718
2667,Post Malone Featuring Quavo,Congratulations,0.730046
15239,Prince,When Doves Cry,1.259932
25813,Ariana Grande,Dangerous Woman,1.548002


In [50]:
# Topsongs data
as_js = 1 # vs json
n = 100
cols = ['artist', 'title', 
        'year',
        'raw', 'icomp', 'rscore']
df = lyrs[cols].sort_values(by='rscore', ascending=0).head(n)
if as_js:
    to_js(df, 'topsongs-data')
else:
    df.to_json(
        os.path.expanduser('~/src/song-repetition/src/assets/topsongs-data.json'),
        orient='records'
    )

Wrote results to topsongs-data.js


In [16]:
# Data for the artist discography widget

disco_dir = 'discogs'
final_dest = '~/src/song-repetition/src/assets/'
def artist_to_fname(a):
    slug = a.replace(' ', '_').replace('/', '').replace("'", "").lower() + '_discog'
    return slug + '.json'

songs_per_artist = lyrs.groupby('artist').size()
stars = songs_per_artist.to_frame(name='nsongs')
istar = stars.index
stars['year'] = lyrs.groupby('artist')['year'].median().loc[istar].astype(int)

irealstar = (
    (stars['year'] > 2000) & (stars['nsongs'] >= 5)
    |
    (stars['year'] > 1990) & (stars['nsongs'] >= 7)
    |
    (stars['year'] > 1980) & (stars['nsongs'] >= 9)
    |
    (stars['nsongs'] >= 15)
)

stars = stars.loc[irealstar]

stars['name'] = stars.index.to_series()
stars['fname'] = stars['name'].apply(artist_to_fname)

starmap = stars['fname'].to_dict()
dict_to_js(starmap, 'starmap.js')

# TODO: should probably rm discogs/* at this point
for star, fname in starmap.iteritems():
    cols = ['title', 'yearf', 'rscore']
    songs = lyrs[lyrs['artist']==star][cols]
    path = os.path.join(disco_dir, fname)
    songs.to_json(path, orient='records')
os.system('cp -r {} {}'.format(disco_dir, final_dest))

Wrote results to starmap.js


0

In [3]:
# Supplemental data for discog widget (dist. of rscores)
pctiles = [.001, .999]
scores = lyrs['rscore']
histrange = [scores.quantile(pct) for pct in pctiles]
hist, edges = np.histogram(lyrs['rscore'].values, bins='auto')#, range=histrange)

histy = []
for i, count in enumerate(hist):
    bar = dict(count=count, left=edges[i], right=edges[i+1])
    histy.append(bar)
dict_to_js(histy, 'histogram-data.js')

Wrote results to histogram-data.js


In [74]:
# Data for the repetition over the years chart
from scipy.stats import hmean
bookends = 2 # how many years to take off front and back
window_size = 3
# TODO: the way I'm calculating rolling averages probably isn't technically 
# correct, since it's an average of averages, and different years have different
# #s of songs.
rolling = 1
# Harmonic mean made sense when using ratios, but since we're now using 'rscores', which are
# in log space, we can just use the arithmetic mean.
use_harmonic_mean = 0
ihit = lyrs['peak'] <= 10
def movingaverage(interval, window_size):
    window = np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

if use_harmonic_mean:
    mean_per_year = lyrs.groupby('year')['rscore'].apply(hmean)
else:
    mean_per_year = lyrs.groupby('year')['rscore'].mean()
if rolling:
    y = movingaverage(mean_per_year.values, window_size)
else:
    y = mean_per_year.values
yrs = mean_per_year.index.values
frame = np.stack([yrs, y]).T
ot = pd.DataFrame(frame, columns=['year', 'rscore'])


if use_harmonic_mean:
    mean_per_year = lyrs[ihit].groupby('year')['rscore'].apply(hmean)
else:
    mean_per_year = lyrs[ihit].groupby('year')['rscore'].mean()
mean_per_year = mean_per_year.loc[yrs]
if rolling:
    y = movingaverage(mean_per_year.values, window_size)
else:
    y = mean_per_year.values
ot['hitsRscore'] = y
ot  = ot.set_index('year')


# Get each year's most repetitive song. I hate pandas.
idm = lyrs.groupby('year')['rscore'].idxmax()
mostrep = lyrs.ix[idm,['year', 'artist', 'title', 'rscore']]
mos = mostrep.set_index('year')
dat = []

for y in yrs[bookends:-bookends]:
    row = ot.loc[y].to_dict()
    row['year'] = int(y) # json can't serialize numpy ints. Blurgh.
    row['topsong'] = mos.loc[y].to_dict()
    dat.append(row)

dict_to_js(dat, 'years.js', varname='DATA')

Wrote results to years.js


In [78]:
ot.tail()

,rscore,hitsRscore
year,,
2013.0,1.144306,1.259210
2014.0,1.153249,1.269777
2015.0,1.151047,1.264940
2016.0,1.139138,1.292910
2017.0,0.754364,0.862035


In [14]:
# Data for the artist repetitiveness chart
min_songs = 15

songs_per_artist = lyrs.groupby('artist').size()
stars = songs_per_artist[songs_per_artist >= min_songs].to_frame(name='nsongs')
istar = stars.index

stars['year'] = lyrs.groupby('artist')['year'].median().loc[istar].astype(int)
stars['rscore'] = lyrs.groupby('artist')['rscore'].median().loc[istar]
stars['name'] = stars.index

# Get each artist's most repetitive song. I hate pandas.
idm = lyrs.groupby('artist')['rscore'].idxmax()
mostrep = lyrs.ix[idm,['artist', 'title', 'rscore']]
mos = pd.Series(data=mostrep['title'].values, index=mostrep['artist'].values)
moss = pd.Series(data=mostrep['rscore'].values, index=mostrep['artist'].values)
stars['mostrep'] = mos[istar]
stars['topscore'] = moss[istar]

print "Got {} stars".format(len(stars))
pprint = 1
json_str = stars.to_json(None, orient='records', lines=0)
if pprint:
    json_str = json_str.replace('},', '},\n  ')
with open('stars.json', 'w') as f:
    f.write('var artists = ')
    f.write(json_str)
    f.write(';\nexport default artists;')
os.system('cp stars.json ~/src/lalala/src/js/artist-data.js')
stars.head()

Got 204 stars


,nsongs,year,rscore,name,mostrep,topscore
artist,,,,,,
ABBA,20,1977,1.083842,ABBA,Voulez-vous,1.719892
Aerosmith,29,1989,0.949182,Aerosmith,Rag Doll,1.579541
Al Green,17,1973,0.814384,Al Green,Full Of Fire,1.527973
Alan Jackson,21,2002,0.896865,Alan Jackson,It Must Be Love,1.755935
Alice Cooper,17,1975,0.954690,Alice Cooper,Clones (We're All),1.495904


In [40]:
type(json_str)

str

In [35]:
mostrep = lyrs.ix[idm,['artist', 'title']]
mos = pd.Series(data=mostrep['title'].values, index=mostrep['artist'].values)

'Til Tuesday                    Coming Up Close
(+44)             When Your Heart Stops Beating
10 Years                              Wasteland
10,000 Maniacs            Candy Everybody Wants
10cc                         Good Morning Judge
dtype: object

In [4]:
#lyrs[lyrs['artist'].str.contains('Gwen Stefani')]
lyrs[lyrs['artist']=='Gwen Stefani']['rscore'].mean()
lyrs[lyrs['artist']=='Gwen Stefani']['year'].median()
lyrs[lyrs['artist']=='Donna Summer'][['rscore', 'title']]

,rscore,title
7116,1.237039,The Wanderer
7117,0.790281,There Goes My Baby
7118,1.376318,Walk Away
7119,1.648183,Last Dance
7120,1.082198,Rumour Has It
7121,1.073694,State Of Independence
7122,0.801985,On The Radio
7124,1.455016,I Will Go With You (Con Te Partiro)
7125,1.600417,She Works Hard For The Money
7128,0.989411,Could It Be Magic
